In [19]:
import os
import pandas as pd
import numpy as np
from scipy import optimize
from pathlib import Path
from pandas import DataFrame

In [20]:
curr_file = os.path.abspath(".")
data_dir = Path(curr_file).parent / "data"
figure_dir = Path(curr_file).parent / "figures"
figure_dir.mkdir(exist_ok=True)

raw_solar = pd.read_csv(
    data_dir / "solar_daily_2014-2019.csv", parse_dates=["time"], index_col="time"
)
wind = pd.read_csv(
    data_dir / "onshore_daily_2014-20.csv", parse_dates=["time"], index_col="time"
)

**We adjust the solar capacities to representative daily values.**

In [21]:
solar = raw_solar / 4

In [22]:
def objective_func(
    w: np.ndarray, C: np.ndarray, E_demand: float, r: np.ndarray
) -> float:
    """Quadprog function to minimize. This function is the objective function for the quadratic programming problem.

    Args:
        w (np.ndarray): Weights of the installed capacity
        C (np.ndarray): Covariance matrix of the locations
        E_demand (float): Expected demand
        r (np.ndarray): Expected capacity factors of the locations

    Returns:
        float: The value of the objective function
    """
    return w.T @ C @ w + (w @ r - E_demand) ** 2


def quadprog(X: DataFrame, E_demand: float) -> tuple[np.ndarray, float]:
    """Find the optimal weights for the installed capacity using quadratic programming.

    Args:
        X (DataFrame): DataFrame of the capacity factors of the locations
        E_demand (float): Expected demand

    Returns:
        np.ndarray: Weights of the installed capacity
    """
    C = X.cov().to_numpy()
    r = X.mean()
    n = len(C)

    # Set random seed to ensure reproducibility
    np.random.seed(42)
    x0 = np.random.rand(n)

    # Constrain w to be greater than 0
    bounds = [(0, None) for i in range(n)]

    results = optimize.minimize(
        objective_func,
        x0,
        args=(C, E_demand, r),
        bounds=bounds,
    )
    w = results.x

    L = objective_func(w, C, E_demand, r)

    return w, L


def array_to_LaTeX(X, x: np.ndarray) -> str:
    df = pd.DataFrame(x, index=X.columns)
    df.loc["Total", :] = df.sum()
    TeX = df.to_latex(header=False, float_format="%.2f")
    # values = TeX.split("\n")[3:-3]
    # bmatrix = "\\begin{bmatrix}\n" + "\n".join(values) + "\n\\end{bmatrix}"
    return TeX

Expected demand of solar energy is $\hat{\mu}_E = 100 \text{GWh}$.

In [23]:
var_std_str = "Minimal production variance is {:.2f}"

w, L = quadprog(solar, 100)
print(array_to_LaTeX(solar, w))

print(var_std_str.format(L))

\begin{tabular}{lr}
\toprule
\midrule
Rome & 206.45 \\
Berlin & 0.00 \\
London & 20.19 \\
Paris & 0.22 \\
Madrid & 164.99 \\
Athens & 291.34 \\
Oslo & 0.00 \\
Total & 683.19 \\
\bottomrule
\end{tabular}

Minimal production variance is 427.38


Expected demand for wind energy in Norway is $\hat{\mu}_E = 1 \text{GWh} = 1 000 \text{MWh}$.

In [24]:
w, L = quadprog(wind, 1)
print(array_to_LaTeX(wind, w))

print(var_std_str.format(L))

\begin{tabular}{lr}
\toprule
\midrule
Utsira Nord & 0.29 \\
Sørlige Nordsjø II & 0.65 \\
Midtfjellet & 0.00 \\
Havøygavlen & 0.59 \\
Total & 1.53 \\
\bottomrule
\end{tabular}

Minimal production variance is 0.14


In [25]:
w, L = quadprog(wind, 1000)
print(array_to_LaTeX(wind, w))

print(var_std_str.format(L))

\begin{tabular}{lr}
\toprule
\midrule
Utsira Nord & 291.66 \\
Sørlige Nordsjø II & 651.48 \\
Midtfjellet & 0.00 \\
Havøygavlen & 591.07 \\
Total & 1534.20 \\
\bottomrule
\end{tabular}

Minimal production variance is 138202.63


In [61]:
def single_objective_func(w: np.ndarray, v: float, E_demand: float, r: float) -> float:
    return w * v * w + (w * r - E_demand) ** 2


def single_installement(X: DataFrame, E_demand: float) -> tuple[np.ndarray, float]:
    """Calculate the variance per location.

    Args:
        X (DataFrame): DataFrame of the capacity factors of the locations
        E_demand (float): Expected demand

    Returns:
        np.ndarray: Variance per location
    """

    V = X.var().to_numpy()
    r = X.mean().to_numpy()
    x0 = np.random.rand(V.size)
    opt_w = np.zeros(V.size)
    single_res = np.zeros(V.size)

    for i, (v, r_, x0_) in enumerate(zip(V, r, x0)):
        bounds = [(0, None)]
        results = optimize.minimize(
            single_objective_func,
            x0_,
            args=(v, E_demand, r_),
            bounds=bounds,
        )
        opt_w[i] = results.x.item()
        single_res[i] = results.fun

    df = pd.DataFrame(
        {"Production variance": single_res, "Installed capacity": opt_w},
        index=X.columns,
    )

    return df


def var_per_location_to_LaTeX(X: DataFrame, E_demand: float) -> str:
    df = single_installement(X, E_demand)
    _, L = quadprog(X, E_demand)

    percent_red = (1 - L / df["Production variance"].min()) * 100

    print(f"Reduction is {percent_red:.2f}% from multi-location.")

    TeX = df.to_latex(float_format="%.2f")
    return TeX

In [62]:
TeX = var_per_location_to_LaTeX(solar, 100)
print(TeX)

Reduction is 41.89% from multi-location.
\begin{tabular}{lrr}
\toprule
 & Production variance & Installed capacity \\
\midrule
Rome & 909.23 & 658.45 \\
Berlin & 2355.29 & 778.94 \\
London & 2383.43 & 792.02 \\
Paris & 2210.35 & 726.85 \\
Madrid & 881.24 & 609.82 \\
Athens & 735.50 & 665.02 \\
Oslo & 3294.55 & 767.10 \\
\bottomrule
\end{tabular}



In [63]:
TeX = var_per_location_to_LaTeX(wind, 1)
print(TeX)

Reduction is 29.14% from multi-location.
\begin{tabular}{lrr}
\toprule
 & Production variance & Installed capacity \\
\midrule
Utsira Nord & 0.25 & 1.31 \\
Sørlige Nordsjø II & 0.20 & 1.26 \\
Midtfjellet & 0.55 & 1.87 \\
Havøygavlen & 0.31 & 1.47 \\
\bottomrule
\end{tabular}



In [50]:
W = np.diag(opt_w)
C = solar.cov().to_numpy()
r = solar.mean().to_numpy()
vec_objective_func = np.vectorize(
    lambda W: objective_func(W, C, 100, r), signature="(n)->()"
)
prod_variance = vec_objective_func(W)

In [51]:
prod_variance

array([ 909.22711306, 2355.29451088, 2383.43040721, 2210.35492799,
        881.24054192,  735.50385809, 3294.55154983])

In [52]:
opt_w * r

array([90.90771772, 76.44705354, 76.16571103, 77.89644926, 91.18759466,
       92.64496236, 67.05453177])